In [16]:
import cx_Oracle
import pandas as pd


db_username_read = open('db_username.txt')
db_username = db_username_read.read()
db_username_read.close()

db_password_read = open('db_password.txt')
db_password = db_password_read.read()
db_password_read.close()

db_host_read = open('db_host.txt')
db_host = db_host_read.read()
db_host_read.close()

connection = cx_Oracle.connect(user=db_username, password=db_password, dsn=db_host)
cursor = connection.cursor()

# Wyciągnięcie danych z bazy
query = """
SELECT gs.game_name, p.platform_name, pub.publisher_name, d.developer_name, g.genre_name, r.rating_name, gs.na_sales, gs.eu_sales, gs.jp_sales, gs.other_sales, gs.critic_score, gs.critic_count, gs.user_score, gs.user_count, gs.release_date
FROM game_sales gs
JOIN platforms p ON gs.platform_id = p.platform_id
JOIN publishers pub ON gs.publisher_id = pub.publisher_id
JOIN developers d ON gs.developer_id = d.developer_id
JOIN genres g ON gs.genre_id = g.genre_id
JOIN ratings r ON gs.rating_id = r.rating_id
"""
cursor.execute(query)

columns = ['Game_Name', 'Platform', 'Publisher', 'Developer', 'Genre', 'Rating', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Critic_Score', 'Critic_Count', 'User_Score', 'User_Count', 'Release_Date']
data = cursor.fetchall()
df = pd.DataFrame(data, columns=columns)

df.head()

,Game_Name,Platform,Publisher,Developer,Genre,Rating,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Release_Date
0,New Super Luigi U,WiiU,Nintendo,Nintendo,Platform,E,1250000,620000,180000,180000,77,59,7.9,288,2013
1,Tetris DS,DS,Nintendo,Nintendo,Puzzle,E,630000,50000,1350000,80000,84,56,8.7,44,2006
2,Classic NES Series: Super Mario Bros.,GBA,Nintendo,Nintendo,Platform,E,0,0,1390000,30000,84,14,8.6,44,2004
3,Pikmin 2,GC,Nintendo,Nintendo,Strategy,E,480000,130000,560000,30000,90,54,9.1,137,2004
4,Mario vs. Donkey Kong,GBA,Nintendo,Nintendo,Puzzle,E,680000,250000,210000,20000,81,43,8.0,31,2004


In [17]:
from scipy import stats
from statsmodels.stats.diagnostic import lilliefors

columns = ['NA_Sales', 'EU_Sales', 'Critic_Score', 'Critic_Count']

# H0 : Zmienne 'NA_Sales', 'EU_Sales', 'Critic_Score', 'Critic_Count' mają rozkład normalny.
# H1 : Zmienne 'NA_Sales', 'EU_Sales', 'Critic_Score', 'Critic_Count' nie mają rozkładu normalnego.

for column in columns:
    shapiro_results = stats.shapiro(df[column])

    ks_results = stats.kstest(df[column], 'norm')

    lilliefors_results = lilliefors(df[column])

    # Print the results
    print(f'Kolumna: {column}')
    print('Test Shapiro-Wilka:', shapiro_results)
    print('Test Kołomogorowa – Smirnowa:', ks_results)
    print('Test Lillieforsa:', lilliefors_results)
    print()
    
print("Pvalue wszystkich zmiennych jest < 0.05, więc odrzucamy hipotezę zerową więc ich rozkład nie jest normalny.")

Kolumna: NA_Sales
Test Shapiro-Wilka: ShapiroResult(statistic=0.32624709606170654, pvalue=0.0)
Test Kołomogorowa – Smirnowa: KstestResult(statistic=0.9204395604395604, pvalue=0.0, statistic_location=10000, statistic_sign=-1)
Test Lillieforsa: (0.34171641057249014, 0.0009999999999998899)

Kolumna: EU_Sales
Test Shapiro-Wilka: ShapiroResult(statistic=0.2929936647415161, pvalue=0.0)
Test Kołomogorowa – Smirnowa: KstestResult(statistic=0.8644688644688645, pvalue=0.0, statistic_location=10000, statistic_sign=-1)
Test Lillieforsa: (0.3656158046873703, 0.0009999999999998899)

Kolumna: Critic_Score
Test Shapiro-Wilka: ShapiroResult(statistic=0.9636805653572083, pvalue=2.761719651146815e-38)
Test Kołomogorowa – Smirnowa: KstestResult(statistic=1.0, pvalue=0.0, statistic_location=13, statistic_sign=-1)
Test Lillieforsa: (0.0786937887537918, 0.0009999999999998899)

Kolumna: Critic_Count
Test Shapiro-Wilka: ShapiroResult(statistic=0.9195757508277893, pvalue=0.0)
Test Kołomogorowa – Smirnowa: Kstes

C:\Python310\lib\site-packages\scipy\stats\_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [18]:
from scipy.stats import f_oneway

# H0 : Nie ma istotnej różnicy między średnią sprzedażą w różnych regionach.
# H1 : Różnica między średnią sprzedażą w różnych regionach jest istotna.

na_sales = df["NA_Sales"]
eu_sales = df["EU_Sales"]
jp_sales = df["JP_Sales"]
other_sales = df["Other_Sales"]

f_stat, f_pvalue = f_oneway(na_sales, eu_sales, jp_sales, other_sales)
print(f"Test ANOVA: Statystyka = {f_stat}, p-wartość = {f_pvalue}")
print("Pvalue jest < 0.05 odrzucamy hipotezę zerową, więc istnieje istotna różnica między średnią sprzedażą w różnych regionach")

Test ANOVA: Statystyka = 414.4237319347391, p-wartość = 2.721379311330356e-263
Pvalue jest < 0.05 odrzucamy hipotezę zerową, więc istnieje istotna różnica między średnią sprzedażą w różnych regionach


In [19]:
from scipy.stats import levene

# H0 : Wariancje liczb sprzedanych kopii gier między regionami są równe.
# H1 : Wariancje liczb sprzedanych kopii gier między regionami nie są równe.

levene_stat, levene_pvalue = levene(na_sales, eu_sales, jp_sales, other_sales)
print(f"Test Levene'a: Statystyka = {levene_stat}, p-value = {levene_pvalue}")
print("P-value jest < 0.05 odrzucamy hipotezę zerową, więc wariancje nie są równe.")

Test Levene'a: Statystyka = 298.93774732362385, p-value = 5.780531871654576e-191
P-value jest < 0.05 odrzucamy hipotezę zerową, więc wariancje nie są równe.


In [20]:
df['Total_Sales'] = na_sales + eu_sales + jp_sales + other_sales

In [21]:
from scipy.stats import pearsonr

# H0: Nie istnieje zależność między liczbą sprzedanych kopii w Ameryce Północnej, a w Japonii.
# H1: Istnieje zależność między liczbą sprzedanych kopii w Ameryce Północnej, a w Japonii.

correlation_stat, p_value = pearsonr(na_sales, jp_sales)
print(f"Korelacja Pearsona: {correlation_stat}, p-value: {p_value}")
print("P-value jest < 0.05 odrzucamy hipotezę zerową, więc istnieje zależność między liczbą sprzedanych kopii w Ameryce Północnej, a w Japonii.")

Korelacja Pearsona: 0.46860734911026697, p-value: 0.0
P-value jest < 0.05 odrzucamy hipotezę zerową, więc istnieje zależność między liczbą sprzedanych kopii w Ameryce Północnej, a w Japonii.


In [22]:
# H0: Nie istnieje zależność między oceną krytyków, ogólną sprzedażą.
# H1: Istnieje zależność między oceną krytyków, ogólną sprzedażą.

correlation_stat, p_value = pearsonr(df['Critic_Score'], df['Total_Sales'])
print(f"Korelacja Pearsona: {correlation_stat}, p-value: {p_value}")
print("P-value jest < 0.05 odrzucamy hipotezę zerową, więc istnieje zależność między oceną krytyków, a ogólną sprzedażą.")

Korelacja Pearsona: 0.23755507999330613, p-value: 3.517599461271442e-88
P-value jest < 0.05 odrzucamy hipotezę zerową, więc istnieje zależność między oceną krytyków, a ogólną sprzedażą.


In [23]:
from scipy.stats import ttest_ind

# H0: Średnia sprzedaż gier nie była większa w 2015 niż 2010.
# H1: Średnia sprzedaż gier była większa w 2015 niż 2010.

sales_2010 = df[df['Release_Date'] == 2010]['Total_Sales']
sales_2015 = df[df['Release_Date'] == 2015]['Total_Sales']

# przeprowadzenie testu t
t_stat, p_val = stats.ttest_ind(sales_2015, sales_2010, alternative='greater')

print(f'T-stat = {t_stat}, p-value= {p_val}')
print("P-value jest > 0.05 nie odrzucamy hipotezy zerowej, więc że nie mamy wystarczających dowodów, aby stwierdzić że średnia sprzedaż była większa w 2015 niż w 2010 roku.")

T-stat = -1.4432463986984838, p-value= 0.9252790630064545
P-value jest > 0.05 nie odrzucamy hipotezy zerowej, więc że nie mamy wystarczających dowodów, aby stwierdzić że średnia sprzedaż była większa w 2015 niż w 2010 roku.


In [24]:
# H0: Średnia sprzedaż gier Nintendo i EA jest równa.
# H1: Średnia sprzedaż gier Nintendo jest większa niż EA.

nintendo_sales = df[df['Publisher'] == 'Nintendo']['Total_Sales']
ea_sales = df[df['Publisher'] == 'Electronic Arts']['Total_Sales']

t_stat, p_val = stats.ttest_ind(nintendo_sales, ea_sales)
print(f'T-stat = {t_stat}, p-value= {p_val}')
print("P-value jest < 0.05 odrzucamy hipotezę zerową, więc przyjmujemy hipotezę że gry Nintendo sprzedają się lepiej niż EA.")

T-stat = 8.550143597586633, p-value= 3.588366891976883e-17
P-value jest < 0.05 odrzucamy hipotezę zerową, więc przyjmujemy hipotezę że gry Nintendo sprzedają się lepiej niż EA.


In [29]:
# H0: Średnie wartości sprzedaży w Ameryce Północnej, a Europie nie różnią się od siebie.
# H1: Średnie wartości sprzedaży w Ameryce Północnej, a Europie różnią się od siebie.

t_stat, p_val = stats.ttest_ind(df['NA_Sales'], df['EU_Sales'])
print(f"t-statystyka: {t_stat}, p-wartość: {p_val}")
print("P-value jest < 0.05 odrzucamy hipotezę zerową, więc średnie wartości ogólnej sprzedaży w Ameryce Północnej, a Europie różnią się od siebie.")

t-statystyka: 11.026800039682623, p-wartość: 3.730990653234972e-28
P-value jest < 0.05 odrzucamy hipotezę zerową, więc średnie wartości ogólnej sprzedaży w Ameryce Północnej, a Europie różnią się od siebie.


In [26]:
#H0 : Średnia sprzedaż w Ameryce Północnej w 2007 roku wynosiła 500 000.
#H1 : Średnia sprzedaż w Ameryce Północnej w 2007 roku nie wynosiła 500 000.

df_2007 = df[df['Release_Date'] == 2007]
mean_sales = df_2007['NA_Sales'].mean()

t_stat, p_val = stats.ttest_1samp(df_2007['NA_Sales'], 500000)
print(f'T-stat = {t_stat}, p-value= {p_val}')
print("P-value jest < 0.05 odrzucamy hipotezę zerową, więc przyjmujemy hipotezę że średnia sprzedaż w Ameryce Północnej w 2007 roku nie wynosiła 500 000.")

T-stat = -3.0434974219539432, p-value= 0.0024427241826905964
P-value jest < 0.05 odrzucamy hipotezę zerową, więc przyjmujemy hipotezę że średnia sprzedaż w Ameryce Północnej w 2007 roku nie wynosiła 500 000.
